In [1]:
%pylab inline
from pypeds import ipeds 
import pandas as pd
from sequence_utils import match_string_lists

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('./name_matches.csv')
a_names = df.a_names.values

In [3]:
YEARS = [2018]
ic = ipeds.IC(years = YEARS)

In [4]:
hd = ipeds.HD(years = YEARS)

In [5]:
hd.extract(), ic.extract()

(None, None)

In [6]:
hd_df = hd.load()
ic_df = ic.load()

In [7]:
ic_df.to_csv('ic_df.csv')

In [8]:
ic_df.head()

,unitid,peo1istr,peo2istr,peo3istr,peo4istr,peo5istr,peo6istr,cntlaffi,pubprime,pubsecon,...,xactcm75,actcm75,xacten25,acten25,xacten75,acten75,xactmt25,actmt25,xactmt75,actmt75
0,100654.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,...,R,19.0,R,14.0,R,20.0,R,15.0,R,18.0
1,100663.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,R,29.0,R,22.0,R,31.0,R,19.0,R,26.0
2,100690.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,...,R,31.0,R,25.0,R,34.0,R,24.0,R,30.0
4,100724.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,...,R,20.0,R,14.0,R,20.0,R,15.0,R,18.0


In [9]:
hd_df.head()

,unitid,instnm,ialias,addr,city,stabbr,zip,fips,obereg,chfnm,...,necta,countycd,countynm,cngdstcd,longitud,latitude,dfrcgid,dfrcuscg,survey_year,fall_year
0,100654.0,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1.0,5.0,"Dr. Andrew Hugine, Jr.",...,-2.0,1089.0,Madison County,105.0,-86.568502,34.783368,119.0,1.0,2018.0,2018.0
1,100663.0,University of Alabama at Birmingham,,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1.0,5.0,Ray L. Watts,...,-2.0,1073.0,Jefferson County,107.0,-86.799345,33.505697,105.0,1.0,2018.0,2018.0
2,100690.0,Amridge University,Southern Christian University |Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1.0,5.0,Michael C.Turner,...,-2.0,1101.0,Montgomery County,102.0,-86.174010,32.362609,137.0,2.0,2018.0,2018.0
3,100706.0,University of Alabama in Huntsville,UAH |University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1.0,5.0,Robert A. Altenkirch,...,-2.0,1089.0,Madison County,105.0,-86.640449,34.724557,109.0,2.0,2018.0,2018.0
4,100724.0,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1.0,5.0,Quinton T. Ross,...,-2.0,1101.0,Montgomery County,107.0,-86.295677,32.364317,127.0,1.0,2018.0,2018.0


In [10]:
_all = ic_df.merge(hd_df, how='inner', on = "unitid")

In [11]:
def col_search(name, df = _all):
    return [col for col in df.columns if name.lower() in col.lower()]

In [12]:
col_search('act')

['xactnum',
 'actnum',
 'xactpct',
 'actpct',
 'xactcm25',
 'actcm25',
 'xactcm75',
 'actcm75',
 'xacten25',
 'acten25',
 'xacten75',
 'acten75',
 'xactmt25',
 'actmt25',
 'xactmt75',
 'actmt75 ',
 'act',
 'cyactive']

In [12]:
import ray
from tqdm import tqdm

In [13]:
ray.init()

2020-10-15 15:50:59,263	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.1.3',
 'raylet_ip_address': '192.168.1.3',
 'redis_address': '192.168.1.3:6379',
 'object_store_address': '/tmp/ray/session_2020-10-15_15-50-58_794104_63500/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-15_15-50-58_794104_63500/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2020-10-15_15-50-58_794104_63500',
 'metrics_export_port': 56613}

In [14]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        s1, s2 = s2, s1

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]/float(len(s1))


@ray.remote
def find_best_match(word, other_word_list):
    best_distance = np.inf
    best_match = ''
    for other_word in other_word_list:
        distance = levenshtein(word, other_word)
        if distance < best_distance:
            best_distance = distance
            best_match = other_word

    return (word, best_match, best_distance)
        

def match_string_lists_parallel(a,b):
    best_matches = [find_best_match.remote(word, b) for word in tqdm(a)]
    return best_matches


In [15]:
match_futures = match_string_lists_parallel(a_names, _all.instnm.values)
matches = ray.get(match_futures)
matches_df = pd.DataFrame.from_records(matches)
matches_df.to_csv('ipeds_matches.csv')

100%|██████████| 502/502 [00:01<00:00, 383.22it/s]


In [109]:
matches

[('North Park University', 'North Park University', 0.0),
 ('Oral Roberts University', 'Oral Roberts University', 0.0),
 ('Lipscomb University', 'Lipscomb University', 0.0),
 ('Saginaw Valley State University', 'Saginaw Valley State University', 0.0),
 ('Clemson University', 'Clemson University', 0.0),
 ('Pfeiffer University', 'Pfeiffer University', 0.0),
 ('Elon University', 'Elon University', 0.0),
 ('Assumption College', 'Assumption College', 0.0),
 ('Canisius College', 'Canisius College', 0.0),
 ('Colorado State University-Pueblo', 'Colorado State University-Pueblo', 0.0),
 ('College of Mount St Joseph', 'College of St Joseph', 0.23076923076923078),
 ('University of Wisconsin-Whitewater',
  'University of Wisconsin-Whitewater',
  0.0),
 ('University of Indianapolis', 'University of Indianapolis', 0.0),
 ('Western Connecticut State University',
  'Western Connecticut State University',
  0.0),
 ('Ottawa University-Ottawa', 'Ottawa University-Ottawa', 0.0),
 ('Houghton College', 'Hou

In [32]:
df

,Unnamed: 0,d_names,a_names,sim_score,admit_rate
0,0,North Park,North Park University,0.816497,0.523353
1,1,Oral Roberts,Oral Roberts University,0.816497,0.435427
2,2,Lipscomb,Lipscomb University,0.707107,0.526969
3,3,Saginaw Valley State,Saginaw Valley State University,0.866025,0.786528
4,4,Clemson,Clemson University,0.707107,0.578703
...,...,...,...,...,...
497,497,Eastern Mennonite,Eastern Mennonite University,0.816497,0.650513
498,498,Dallas Baptist,Dallas Baptist University,0.816497,0.423033
499,499,Manchester University,Manchester University,1.000000,0.711617
500,500,George Washington,George Washington University,0.816497,0.330805


In [70]:
act_data = _all[['acten25', 'acten75', 'instnm']]

w_act = act_data.merge(df, how='inner', left_on='instnm', right_on='a_names')

w_act['actmed'] = (w_act.acten25 + w_act.acten75)/2

combined = w_act.merge(pd.read_csv('./data.csv'), left_on = 'd_names', right_on = 'College').dropna()

In [71]:
X = pd.get_dummies(combined[['actmed',
 'Position',
 'Class', 
 'Conference', 
 'latitude', 
 'longitude', 
 #'cluster'
]])



In [110]:
X.shape

(9495, 105)

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [91]:
le = LabelEncoder()
y = le.fit_transform(combined.College)
model = LogisticRegression(verbose = 2, max_iter = 300)
model.fit(X,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/mgb/anaconda3/envs/flask/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.8s finished


LogisticRegression(max_iter=300, verbose=2)

In [96]:
sorted(np.exp(model.predict_log_proba(X))[0])

[7.929296460293048e-65,
 3.9774417015972937e-56,
 8.981292360027074e-55,
 1.610121914814996e-54,
 1.0564311985446863e-51,
 9.090128630694996e-50,
 1.309303057255402e-47,
 1.4976675332429246e-46,
 4.600091532091733e-46,
 6.177526207767655e-46,
 6.9327697295540576e-46,
 3.546781271304643e-45,
 6.266588600198898e-45,
 6.493852934232906e-45,
 2.6523158747085335e-43,
 1.0440754028320261e-42,
 3.027295401574966e-42,
 4.07961425333462e-42,
 5.601066732279579e-42,
 8.145293200992152e-40,
 2.2302057655790037e-39,
 3.0637438525757526e-38,
 7.910127824653287e-35,
 3.0684050363700017e-34,
 9.241216904126487e-32,
 1.3588842397905298e-31,
 4.587481362953912e-31,
 1.5304487226947647e-29,
 5.348565229230682e-29,
 1.6163406364756386e-28,
 1.7717523693211617e-27,
 1.8501792112744754e-27,
 3.476503149755451e-27,
 7.92904424781093e-27,
 1.4000470650488028e-26,
 2.6329592915311474e-26,
 8.464477789398821e-26,
 9.869184663274748e-26,
 4.072683263084624e-23,
 6.889430302800297e-23,
 8.931448462422893e-23,
 2

In [97]:
from sklearn.naive_bayes import GaussianNB

In [98]:
bayes = GaussianNB()

In [105]:
bayes.fit(X, y).predict_proba(X)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])